In [2]:
import cv2
import os
from time import time
import glob
import h5py
import pandas as pd
from math import radians, degrees, atan2, sqrt

In [3]:
os.chdir("/home/donghan/DeepLabCut/data/")
filenames = glob.glob('*.h5') 
#Return the file name with extention of .h5, which contain the data of coordination axis
f = []
for filename in filenames:
    f = h5py.File(filename, 'r')
    start = filename.find('10') 
    #Find the string that start with "10"
    end = filename.find(' rotated', start) 
    #Return the string with end of " rotated", aims to name the file
    csvfile = []
    with pd.HDFStore(filename, 'r') as d:
        df = d.get(list(f.keys())[0])
        df.to_csv(filename[start:end] + '.csv') 
        #Automaticaly change to unique file name with specific mouse number
        csvfile.append(filename[start:end] + '.csv')
for i in csvfile:
    data = pd.read_csv(i, skiprows = 2) 
    #Skip the rows of scorer and bodyparts
    move_data = data.loc[300:] 

In [ ]:
def videoWriter(task, filename, outputName, corrX, corrY, distX, distY, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass
    

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (distY, distX)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out
    
def videoCropper(filename, outputName, corrX, corrY, distX, distY, displayVideo = False):

    '''
    Task: Crop videos to actural mouse moving edge
    
    PARAMETERS:
    video: file that await for writting
    
    filename: video that need to be trim
    
    outputName: filename of output video
    
    corrX: coordinates of the start point in X
    
    corrY: coordinates of the start point in Y
    
    dist: cropped video is a square, as long as the coordinates of the start point is defined, the other
    three vertex points are clear. 

    '''
    # Calculate the other three points' coordinate
    corrX1 = corrX + distX
    corrY1 = corrY + distY

    video = videoWriter("edited", filename, outputName, corrX, corrY, distX, distY, displayVideo)
    cap = video[0] 
    out = video[1]                  
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    # Not capturing a video
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    # Read videos and rotate by certain degrees
    while(cap.isOpened()):
        # Flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = img[corrX:corrX1, corrY:corrY1]
            out.write(img2)
            if displayVideo == True:
                cv2.imshow('edited video',img2) 
            if count == frameCnt:
                print (filename, 'successfully ', "edited!")
                break
            else:
                count += 1
            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    

    
def getCoord(data):
    '''
    Task: Obtaining the coordinates of cropped points
    
    data: DataFrame
    Mouse moving location, by pixel
    
    !!! We are assuming that the mouse at least reaches the boundary once
    '''
    x = sorted(data['x'], reverse = True)
    y = sorted(data['y'], reverse = True)
    
    # Try to exclude outliers
    for i in range(10):
        xMax = round(x[i + 1]) if x[i] - x[i + 1] >= 1 else round(x[i])
        
        xMin = round(x[-i-1]) if x[-i] - x[-i-1] >= 1 else round(x[-i])
            
        yMax = round(y[i + 1]) if y[i] - y[i + 1] >= 1 else round(y[i])

        yMin = round(y[-i-1]) if y[-i] - y[-i-1] >= 1 else round(y[-i])
    
    lengthX = xMax - xMin
    lengthY = yMax - yMin
    
    
    return xMin, yMin, lengthX, lengthY



def dataModifier(data):
    '''
    Task: Modify the location data based on the cropped video
    
    data: DataFrame
    '''
    
    x, y, lengthX, lengthY = getCoord(data)
    filtered = data.copy(deep = True)
    filtered = filtered[(filtered['y']>x) & (filtered['x']>y)]
    filtered['x'] = filtered['x'] - y
    filtered['y'] = filtered['y'] - x
    filtered['coords'] = list(range(1,len(filtered)+1))
    filtered.index = list(range(1,len(filtered)+1))
    
    return filtered

In [ ]:
def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    return (direction, degreeL)
# Whether the mouse's direction towards bullying mouse or not, 1: yes; 0: no
# The degree of head direction, clockwise.


## Calculate Angle and Distance between enclosure and defeated mouse

In [94]:
def quadrant(a, b): 
    '''
    Task: Check if two points are in the same quadrant
    '''
    if ((a[0] > 0 and b[0] > 0) or (a[0] < 0 and b[0] < 0)) and ((a[1] > 0 and b[1] > 0) or (a[1] < 0 and b[1] < 0)):
            return True
    else:
        return False
    
def getAngle(a, b, c):
    '''
    Task: Calculate angle between three points. 
    
    PARAMETERS
    -----------
    a, b, c: tuple
        Three points in the coordinate system
    '''
    cb = atan2(c[1]-b[1], c[0]-b[0])
    ab = atan2(a[1]-b[1], a[0]-b[0])

    # If they are in the same quadrant: acute angle
    if quadrant(c, b) and quadrant(a, b):
        angle = degrees(cb - ab)
    else:
        # Obtuse angle or in different quadrant
        rad360 = radians(360)
        # cd or ab < 0 means they over 180 degree
        # +ran360 produce a positive radian representing degrees over 180
        cb = rad360 + cb if cb < 0 else cb
        ab = rad360 + ab if ab < 0 else ab
        # Order of points may changed in a motional situation, if exceed 180, adjust it by 360-
        angle = 360 - abs(degrees(cb - ab)) if abs(degrees(cb - ab)) > 180 else abs(degrees(cb - ab))
            
    return angle + 360 if angle < 0 else angle

def getDist(a, b):
    '''
    Task: Calculate distance
    '''
    dist = sqrt((b[0] - a[0])**2 + (b[1] - a[1])**2)  
    return dist  

In [73]:
def calDist_Angle(data, enclosure):
    '''
    Task: Calculate distance and angle between enclosure and defeated mouse
    reference point is the center point of head of tail regarding Angle
    head point for Distance
    
    Parameters
    ----------
    data: DataFrame
        Read location data
    
    enclosure: tuple
        enclosure's coordinate, ex. (68,204)
    '''
    head = pd.concat([data["x"], data["y"]],axis=1)
    tail = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    # Head--------------------Tail-----------
    mid = (head.x + tail.x) / 2, (head.y + tail.y) / 2
    # Avoid Pandas KeyError
    mid = np.array(mid)
    x = np.array(head.x)
    y = np.array(head.y)
    
    angle = []
    dist = []
    for i in range(len(head)):
        # Set middle point of the mouse to be (0,0), then calculate the corresponding relative converted coordinates
        # Order: head relative location, central point, enclosure relative location
        angle.append(getAngle((x[i] - mid[0][i], y[i] - mid[1][i]),(0, 0), (enclosure[0] - mid[0][i], enclosure[1] - mid[1][i])))
        dist.append(getDist((mid[0][i], mid[1][i]), enclosure))
    return angle, dist

## Coordinate System Transformation

In [105]:
import numpy as np
import cv2
 
def orderPoints(pts):
    '''
    Task: sort points by the following order: top-left, top-right, bottom-left, bottom-left
    
    PARAMETERS:
    -----------
    pts: list 
        list of tuple of points coordinate
    '''
    
    rect = np.zeros((4, 2), dtype = "float32")

    # the top-left point will have the smallest sum, whereas
    # the bottom-right point will have the largest sum
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # now, compute the difference between the points, the
    # top-right point will have the smallest difference,
    # whereas the bottom-left will have the largest difference
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect

def fourPointTransform(pts, image = None):
    '''
    Task: Transform image by its corner four points. 
    
    PARAMETERS:
    -----------
    image: array, Optional if intend to transform an image
        image matrix
    
    pts: list
        list of tuple of four corner points
    '''
    rect = orderPoints(pts)
    (tl, tr, br, bl) = rect

    # Transformed width
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    # Transformed height
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))
    
    # Specify the destination points based on the dimensions: top-left, top-right, bottom-right, and bottom-left
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # compute the perspective transform matrix and then apply it
    M = cv2.getPerspectiveTransform(rect, dst)
    if image is not None:
        warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
        return warped, M
    return M



In [130]:
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
img = cv2.imread("test2.jpg")
cv2.imwrite("test10.jpg",fourPointTransform(img, pts)[0])

True

In [135]:
def locDataConvert(data, pts):
    '''
    Task: Convert mouse location data to prospective coordicates after correcting coord system.
    
    PARAMETERS:
    -----------
    pts: list
        list of four corner points of the video
    '''
    # Transformation matrix
    matrix = fourPointTransform(pts)
    # x, y, 1
    head = pd.concat([
        data["x"], 
        data["y"], 
        pd.DataFrame([1]*len(data))],axis = 1).values.transpose()
    tail = pd.concat([
        data["x.1"], 
        data["y.1"], 
        pd.DataFrame([1]*len(data))], axis = 1, keys=['x', 'y', 'constant']).values.transpose()
    
    # Dot product of transformed matrix and head, tail data
    transformedHead = np.dot(matrix, head)
    transformedTail = np.dot(matrix, tail)
    
    tHeads = []
    tTails = []
    for i in range(transformedHead.shape[1]):
        tempx = []
        tempy = []
        for x,y in zip(transformedHead, transformedTail):
            tempx.append(x[i])
            tempy.append(y[i])
        tHeads.append(tempx[:2])
        tTails.append(tempy[:2])
    
    return tHeads, tTails

In [131]:
for p in i:
    print(p)

[239.5114147540263, 410.0215598784492, 0.7687282287273324]
[253.58444120196714, 411.5592877616764, 0.7628687858648031]
[-79.24305769822806, 385.8102816363677, 0.8983081233627928]
[-79.45472755845108, 385.60873443300443, 0.8984489592759097]
[280.20231389672153, 410.91553899509404, 0.7528355049977586]
[253.52666357389376, 411.02083127926903, 0.7630500380321306]
[-78.94368057624192, 385.49642611474206, 0.8982858513796199]
[-79.60916674821604, 385.5528947353368, 0.8985247717125633]
[-50.38616527143955, 392.5454525402606, 0.8852351318539069]
[-79.49800571994041, 385.3546075867187, 0.8985406510747538]
[253.71241397361098, 411.1625086397798, 0.7629368429546073]
[-79.88241329340676, 385.19229524856144, 0.8987362423945795]
[-79.4091953412561, 385.5391710287169, 0.8984520203018685]
[-79.38751755471671, 385.38151346235696, 0.8984902665272867]
[-78.83370076381362, 385.4917810462549, 0.8982449822757098]
[-79.34925498468182, 385.3982129803453, 0.898470637502019]
[-78.80628998642119, 385.100442290687

[48.38440093718059, 74.47775984654601, 0.9412567927329818]
[52.918552779643335, 69.40410368867975, 0.9410140720753597]
[59.835833322298356, 68.00912283222135, 0.9387693486430355]
[63.9972292714145, 68.10086418688226, 0.9371439309292375]
[72.47914395838967, 68.11436467193101, 0.9338821923005435]
[81.52521744010393, 67.18578514178537, 0.930682062241879]
[92.70141394866916, 67.95558882179105, 0.926162085468379]
[103.92445142850693, 70.66318759089134, 0.921051690457218]
[113.09575474782947, 69.92624692006726, 0.9177468514793811]
[122.72037111226633, 70.54800235128147, 0.9138665423421508]
[133.8958332693424, 71.4452398291991, 0.9093092034920834]
[146.37595537124355, 76.16971597996519, 0.9031201964453027]
[159.16600206325563, 79.3931132613233, 0.8972555738125605]
[166.65752759285846, 78.01091834108001, 0.8947865162460786]
[176.82581312084466, 78.77758827187301, 0.8906545856291785]
[189.64611706359926, 81.21742609601691, 0.8850098063257635]
[202.8171866835031, 85.2172076393314, 0.878769494622

[14.486327800919987, 245.0482424629184, 0.9038896650662079]
[14.172835836865254, 243.44625024548256, 0.9044833024917601]
[13.996523508727854, 244.062084851052, 0.9043691024485184]
[13.181970222308387, 243.2402647217895, 0.9049247245372871]
[12.491768056134248, 243.87272294546958, 0.9050029896784831]
[13.0721995279332, 244.35527375072908, 0.9046375102069903]
[13.575297922551322, 243.76099648756545, 0.9046198295415543]
[13.554816917687972, 241.57486118614898, 0.905273483448166]
[14.386727534182057, 242.28450174460144, 0.9047443327287861]
[14.44781038014159, 238.5763419759813, 0.9058162676535699]
[18.20246498905503, 234.74760851659246, 0.905505185873804]
[20.598482714167375, 230.51650291601706, 0.9058347933460386]
[22.586652512552263, 227.78586591504708, 0.9058778072675854]
[25.206732252048383, 220.27402340664347, 0.9070904909736253]
[26.5340668890274, 217.70728301747056, 0.9073389044752018]
[26.490686974545824, 214.4263642268483, 0.9083247540623512]
[27.741609068305493, 204.0571904946822

[89.59783607798458, 261.97513510989484, 0.8700404599276699]
[89.6505554501275, 264.17212413991535, 0.8693712176381148]
[90.76829275484467, 263.07798358903824, 0.8692651255809358]
[90.67845015943546, 263.80923041601073, 0.8690836211350579]
[88.2486668814713, 263.1706133266784, 0.8702055055108071]
[88.7654918526476, 263.20355256105375, 0.8699972720565051]
[89.99549612867278, 262.2012124328548, 0.8698209423016879]
[88.6588287515085, 262.66190839484375, 0.8701982418993923]
[87.92692345467893, 264.64898167685783, 0.8698923692190752]
[88.21628607466167, 264.6003702334305, 0.869795590142294]
[89.70271395199205, 265.07086554918783, 0.8690856949887497]
[89.8763251201768, 263.0872593679407, 0.8696049741954535]
[90.5976177023839, 264.5926579697988, 0.8688832417954446]
[88.71132000463624, 264.02400289794787, 0.8697757162739117]
[80.86758971829043, 263.9752949666619, 0.8728027401064131]
[76.5482701804077, 259.32169443764263, 0.8758363932055332]
[76.06170796265336, 256.9963449770343, 0.8767101845695

In [137]:
start = time()
pts = np.array([(28,39),(74,387),(364,23),(377,336)])
i = locDataConvert(data,pts)[0]
[p[0] for p in i]

[239.5114147540263,
 253.58444120196714,
 -79.24305769822806,
 -79.45472755845108,
 280.20231389672153,
 253.52666357389376,
 -78.94368057624192,
 -79.60916674821604,
 -50.38616527143955,
 -79.49800571994041,
 253.71241397361098,
 -79.88241329340676,
 -79.4091953412561,
 -79.38751755471671,
 -78.83370076381362,
 -79.34925498468182,
 -78.80628998642119,
 -79.26622412257629,
 252.99186700777432,
 -79.28629500570028,
 -79.16303952434193,
 253.61228822144307,
 254.01733965346818,
 -79.17970385347736,
 -79.38358664786847,
 -78.78060550235409,
 -79.07834930924501,
 -79.35748208712089,
 254.24306128435188,
 -78.87526657652734,
 -79.38533035136116,
 -79.56381527458757,
 -78.90103097293206,
 -79.06231903284908,
 -79.06930081820893,
 -79.08033779435304,
 -78.69696879252346,
 -79.26250589328765,
 -78.91404979937752,
 -79.1016043220803,
 -78.8363272550281,
 -79.06432603707492,
 -78.60031621640101,
 -78.85856342586007,
 -79.27579657674511,
 -78.96483145898132,
 184.58699579149066,
 -79.010224119075

In [ ]:
csvfile

In [ ]:
test = move_data
test

In [ ]:
import numpy as np
pd.options.mode.chained_assignment = None
p['coords'] = list(range(1,len(p)+1))
p.index = list(range(1,len(p)+1))

In [ ]:
p

In [ ]:
p = test[(test['y']>54) & (test['x']>38)]

In [ ]:
test = move_data['x'] - 38
test[test>0]

In [160]:
def videoWriter(task, filename, outputName, corrX = None, corrY = None, dist = 320, displayVideo = False):
    
    '''
    Task: Takes video as input and returns its capturing and output file. 
    
    If task is edited, then corrX and corrY is the vertex of the cropped video edge. 
    '''

    # Capture video
    cap = cv2.VideoCapture(filename)

    # Read video frame by frame
    # Extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # Make a directory to store the processed videos
    path = "./" + task
    try:  
        os.mkdir(path)
        print ("Successfully created the directory %s " % path)
    except OSError:  
        pass

    #Automatically name the processed videos  
    file = "./" + task + "/" + outputName
    if task == "edited":
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, (dist, dist)) 
        # Another option: cv2.VideoWriter_fourcc(*'XVID')
    else:
        out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz)

    return cap,out

def fieldOfView(data, filename, outputName, radius, rotationAngle, color, viewRange, thickness, displayVideo = False):
    '''    
    Task: draw sector for field of view

    All input parameters must be Int

    PARAMETERS:

    data: mouse movement that read from .h5 file, including head and tail. 

    angle: 0, rotation angle

    radius: radius of the sector, millimeter

    color: sector line color

    viewRange: range that the mouse has vision on

    thickness: the thickness of the sector line

    '''

    #Obtain coordinates data for head and tail, as well as the head direction by degrees
    

#     head = zip(round(data['x']).astype(int), round(data['y']).astype(int))
#     tail = zip(round(data['x.1']).astype(int), round(data['y.1']).astype(int))
    
    
    
#     degrees = head_dir(data)[1]
    
    #Get pixel to mm conversion
#     x, y, lengthX, lengthY = getCoord(data)
#     length = lengthX if lengthX < lengthY else lengthY
#     convert = 440/length # video side length 440mm, square
#     radius = round(radius / convert)
    
    
    #cap: Read video
    #out: Write video
    video = videoWriter("field of view", filename, outputName, displayVideo)
    cap = video[0]
    out = video[1]

    # Get the frames count of video in order to break the loop when it reaches the last frame
    frameCnt = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if (cap.isOpened() == False): 
          print("Unable to read video")
    count = 1
    while(cap.isOpened()):
        try:
            print(1)
            for center, angle in zip(head, degrees):
                ret,img = cap.read()
            #draw the sector representing the field of view of mouse
                angle = int(round(angle))
                startAngle = angle - viewRange
                endAngle = angle + viewRange
                img2 = drawSector(img, radius, tuple(center), rotationAngle, startAngle, endAngle, color, thickness)
                out.write(img2)
                    
                if displayVideo == True:
                    cv2.imshow('Draw field of view video',img2) 
                    
                # Break the loop when the last frame has been rotated
                if count == frameCnt:
                    print (filename, 'successfully ', "added field of view!!!")
                    return
                else:
                    count += 1

                k=cv2.waitKey(30) & 0xff
                #once you inter Esc capturing will stop
                if k==27:
                    break
        except:
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()


In [159]:
tuple([1,1])

(1, 1)

In [147]:
degrees = head_dir(data)[1]


In [151]:
os.chdir("/home/donghan/DeepLabCut/data/")

In [155]:
for i, j in zip(locDataConvert(data,pts)[0], degrees):
    print(i, j)

[239.5114147540263, 410.0215598784492] 57.52367714634317
[253.58444120196714, 411.5592877616764] 56.04611636540828
[-79.24305769822806, 385.8102816363677] 101.68006308988296
[-79.45472755845108, 385.60873443300443] 101.58265166349857
[280.20231389672153, 410.91553899509404] 53.337695325663354
[253.52666357389376, 411.02083127926903] 55.94378192509814
[-78.94368057624192, 385.49642611474206] 101.49199601412181
[-79.60916674821604, 385.5528947353368] 101.85919120191794
[-50.38616527143955, 392.5454525402606] 97.1525912792811
[-79.49800571994041, 385.3546075867187] 101.9793849252414
[253.71241397361098, 411.1625086397798] 56.142947048951264
[-79.88241329340676, 385.19229524856144] 101.99163391325682
[-79.4091953412561, 385.5391710287169] 101.77222914674782
[-79.38751755471671, 385.38151346235696] 101.90094569304478
[-78.83370076381362, 385.4917810462549] 109.20680236340684
[-79.34925498468182, 385.3982129803453] 109.30024813794921
[-78.80628998642119, 385.10044229068774] 135.8607495922317

[18.87809210168645, 195.13104110322203] -107.8660327503548
[19.893158888179926, 194.21307060074437] -106.77151200783817
[20.289256614315015, 190.12369178596938] -105.76370804944638
[20.863163269817825, 190.0724616806387] -105.46943791517413
[20.96948092640389, 188.99249348573935] -105.14531095062989
[23.318304124719212, 189.72653265477996] -101.88265837042674
[22.786163673051213, 195.81524741835545] -103.46694193897214
[22.855573341956784, 196.1815473019984] -103.29156059480452
[22.358817488913644, 197.63624979864443] -103.81460756467301
[22.724263816427786, 197.47026088029799] -103.95033761575854
[21.105765084483473, 198.73856801801256] -106.32885300340227
[21.9502005196808, 197.32375171054034] -105.33489758108526
[21.87943592120952, 196.74757893601915] -104.67222489788435
[23.371474343669178, 200.18693055567903] -103.10959494557545
[25.90355435388519, 204.26267390993746] -101.25486406564053
[30.992732867518235, 216.26231073472138] -90.54243695901438
[37.58450273883767, 224.8082626841

[34.71436913450951, 107.62246869707246] 71.06968429044494
[34.97693619539698, 105.56735691726692] 55.95426603991011
[35.64713425248276, 106.76274397146392] 69.1922657350778
[35.98062590303643, 105.9992149761768] 58.55664280964589
[34.77760318449187, 105.94560816567738] 59.84039524152082
[34.51727615445301, 105.9553003236158] 55.72478017384176
[33.92736311518054, 103.57878953910142] 70.47986697587426
[36.82661470696404, 97.84319586510514] 51.97992543204623
[46.071238979286605, 89.1638981823449] 36.29775136863203
[48.49467897991845, 83.88330126240314] 28.635843932721663
[57.506093127141284, 82.39016585238039] 14.661886172158297
[61.671823402500436, 84.29181345330178] 12.161954436597835
[63.68225724944959, 81.76016736229752] 6.575869887852092
[74.58039137423901, 85.522282932428] 7.577808896163474
[80.66203500988904, 83.29638333399242] 6.316381007162371
[90.1443215887666, 85.35994674997625] -1.8117755509826519
[97.55468222006478, 85.81318601595467] 11.371499923501029
[105.04106288045726, 8

[91.13833360214802, 210.356472120086] -10.640692164817322
[97.86891099369346, 208.79991693238458] -16.995501661539862
[106.61553135008988, 206.25883351902357] -17.390389395152706
[114.87345885188658, 200.79128761190648] -22.31928537992801
[123.91134798177636, 198.20775583125987] -20.016845277729765
[135.45677353463418, 198.40173956686078] -12.73896977083497
[146.60151323545222, 194.62229670183524] -16.200747235344934
[152.7189065571108, 191.52417733483767] -25.05356014702922
[158.51978704281896, 186.8990779924855] -23.165274706574195
[169.44320427303296, 184.27662834499213] -14.358428845976752
[179.1406352746622, 182.61687853175738] -15.541171430949555
[184.0962157457471, 182.8659788008443] -15.482337161648669
[196.82315613563287, 177.35031356766012] -22.994967691190496
[205.92597800530928, 172.69574825092877] -23.493520395220713
[213.52713551895738, 171.87694863793172] -13.532642787341944
[219.09051732911934, 168.4099907650755] -15.300785180570841
[226.58093938307496, 165.179853549066

[22.363865284808085, 200.9169875003569] -96.44791112130488
[22.997533394778678, 201.91585455409393] -96.0405263972095
[22.745829888759857, 200.38062905397032] -96.45428958338485
[22.722509868282174, 200.74434114758398] -95.60488330680634
[22.727738900843732, 201.36680264812128] -96.33638746538129
[23.223322540253143, 202.12427653378913] -94.46644796538746
[22.367218135598847, 202.56505597201738] -94.24528127670267
[21.850961672168864, 203.2392344209326] -94.0840717174968
[21.4168070260723, 204.33753249944564] -92.56464347358
[21.378591970843047, 204.72351951595192] -87.4841180896972
[22.898094299931316, 202.85265197861827] -88.53764950838061
[22.98151528878887, 202.454032834975] -89.0410633957292
[23.79667143797903, 201.92271025953394] -87.7987640157419
[23.35280024619877, 201.90081566146827] -90.52220591337618
[23.20896786551146, 202.01772610526447] -90.90946330069589
[24.666883449361343, 201.3436370737157] -88.42190689354169
[26.814976870382957, 202.19354191468688] -79.64505495017183

[221.3031752974237, 236.84413303978786] -24.879283081651444
[225.22708489670026, 231.86230983239759] -30.50872619355544
[227.5523215564903, 230.50612529771914] -31.276962662548673
[232.54773872173814, 229.74697710591104] -30.659780943255075
[238.30411610864678, 226.94070561790014] -29.589633831973753
[239.31355543763124, 222.24871313942637] -24.870877945512284
[245.32373900582144, 218.36340044574558] -28.527449972049887
[248.68631876629286, 215.59630003957912] -35.14979968997105
[252.9432035549395, 209.72681252139293] -41.27685327512238
[252.03681733502805, 207.87335622369423] -50.24163753727552
[262.67323828721476, 204.44593632079767] -40.362184056948415
[263.8175910965689, 202.70795107018887] -35.90916937390627
[267.7703270322912, 196.6863227800544] -40.982083654164285
[270.1927076438864, 194.00383978931052] -52.38153563400762
[274.4703799438686, 190.74630551428473] -52.777170479318706
[273.4794033938202, 191.79300940003418] -48.22928702166352
[272.5003259538789, 191.89856797179203] 

[98.72137709202566, 263.1427639505329] 112.29720340759104
[97.67398552079585, 263.31008706140796] 113.53693322075027
[98.80222304189793, 263.54595285598987] 111.91384745995661
[98.4897529508596, 264.1958673675029] 111.94301405705446
[97.9567031062551, 263.61191606904424] 112.87500650424894
[98.29759956911194, 263.1073764970374] 113.49053220913831
[97.96470303624322, 263.1864165095926] 114.08046709453312
[98.02142486047312, 264.5497413733065] 112.40331057124267
[98.50264310894046, 264.27051282496416] 112.1051501257044
[98.3528401497906, 264.960652825342] 112.38274580124603
[97.86000432312683, 264.21801945921106] 113.09123397179256
[97.78240953047172, 265.18258413230296] 111.58480192663136
[98.86855225732808, 264.7912424577282] 110.72091278824676
[100.55118606147552, 264.7927851780277] 108.08820589240757
[97.7439040890004, 263.03592659154157] 112.1585147636919
[89.95393242343728, 263.9777386054865] 121.67006195768738
[79.53698004984723, 259.632266183918] 135.7653837260049
[68.39528260601

In [161]:
fieldOfView(locDataConvert(data,pts)[0],"1034 SI_B, Aug 15, 13 7 49.mp4", "transformTest.mp4", 50, 0, BLACK, 60, 1)

1


In [156]:
BLACK = (0,0,0)
from math import degrees, atan2
def drawSector(image, radius, center, angle, startAngle, endAngle, color, thickness):
    # Ellipse parameters
    axes = (radius, radius)

    cv2.ellipse(image, center, axes, angle, startAngle, endAngle, color, thickness)
    points = cv2.ellipse2Poly(center, axes, angle, startAngle, endAngle, thickness)
    point1 = tuple(points[0]) # End's coordinates
    point2 = tuple(points[-1]) # Another end's coordinates
    cv2.line(image, center, point1, color)
    cv2.line(image, center, point2, color)
    return image

def head_dir(data):
    p1 = pd.concat([data["x"], data["y"]],axis=1)
    p2 = pd.concat([data["x.1"], data["y.1"]], axis = 1, keys=['x', 'y']) 
    #Reassign column names
    xDiff = p1.x - p2.x
    yDiff = p1.y - p2.y
    direction = []
    degreeL = []
    for i in range(0,len(xDiff)):
        degree = degrees(atan2(tuple(yDiff)[i], tuple(xDiff)[i]))
#         if degree < 0:
#             degree += 360
#             degreeL.append(degree)
#         else:
        degreeL.append(degree)
        if (degree >= 90 and degree <= 180) or (degree <= -90 and degree >= -180): 
            #Facing encloser
            direction.append(1)
        else: 
            direction.append(0) 
            #Facing other side
    if __name__ == "__main__": 
        return (direction, degreeL)

In [ ]:
x = dataModifier(move_data)

In [ ]:
fieldOfView(x, '1034 SI_B, Aug 15, 13 7 49.mp4', "cropTest1.mp4", 50, 0, BLACK, 60, 1)

In [ ]:
x, y, lengthX, lengthY = getCoord(move_data)

In [ ]:
x, y, lengthX, lengthY

In [ ]:
convertX = 440/lengthX
convertY = 440/lengthY
print(convertX, convertY)

In [ ]:
440/331

In [ ]:
videoCropper('1034 SI_B, Aug 15, 13 7 49 rotated.mp4', "cropTest rotated.mp4", y, x, lengthY, lengthX)

In [ ]:
##Question: choose length of the edge in order to be converted in video processing